In [138]:
import requests
import requests.auth
from requests_oauthlib import OAuth1
import oauth2 as oauth
import time
import os
import json
import pprint
import csv
import base64

In [139]:
# Import credentials from local source:
reader = csv.reader(open("/Users/mchifala/Desktop/ATLS_5412/Credentials.csv"))
credentials = {}
for line in reader:
    credentials[line[0]] = line[1]
    

In [ ]:
# Get a Reddit credentials:
def getRedditHeaders():
    reddit_id = credentials['reddit_id']
    reddit_secret = credentials['reddit_secret']
    reddit_username = credentials['reddit_username']
    reddit_password = credentials['reddit_password']
    reddit_user_agent = credentials['reddit_user_agent']

    reddit_auth = requests.auth.HTTPBasicAuth(reddit_id, reddit_secret)
    reddit_post_data = {'grant_type': 'password', 'username': reddit_username, 'password': reddit_password}
    reddit_token_headers = {'User-Agent': reddit_user_agent}
    r_auth = requests.post("https://www.reddit.com/api/v1/access_token", auth= reddit_auth, data=reddit_post_data, headers=reddit_token_headers)

    reddit_token = r_auth.json()['token_type'] + ' ' + r_auth.json()['access_token']
    
    # Return header dictionary
    return {"Authorization": reddit_token, "User-Agent": reddit_user_agent, 'Content-Type': 'application/json'}


In [140]:
# Get Twitter credentials:
def getTwitterAuth():
    twitter_key = credentials['twitter_key']
    twitter_secret =  credentials['twitter_secret']
    twitter_token = credentials['twitter_token']
    twitter_secret_token = credentials['twitter_secret_token']
    
    #Return authorization object
    return OAuth1(twitter_key, twitter_secret, twitter_token, twitter_secret_token)

getTwitterAuth()

In [141]:
# Get all available Twitter trends:
twitter_auth = getTwitterAuth()
r_trends = requests.get('https://api.twitter.com/1.1/trends/available.json', auth = twitter_auth)

In [142]:
# Extract the location ID codes for trends:
world_codes = []
for i in range(0,len(r_trends.json())):
    world_codes.append(r_trends.json()[i]['woeid'])

test_codes = world_codes[20:30]
print(test_codes)

[26042, 26062, 26734, 28218, 28869, 30079, 30720, 32185, 32452, 32566]


In [143]:
#Search each location ID for its specific trends:
topics = []
for code in world_codes:
    r_trends2 = requests.get('https://api.twitter.com/1.1/trends/place.json?id='+str(code), auth = twitter_auth)
    for i in range(0,len(r_trends2.json()[0])):
        print(r_trends2.json()[0]['trends'][i]['name'])
        topics.append((r_trends2.json()[0]['trends'][i]['name']))

#ARSMUN
Arsenal
#COYG
De Gea
#ARSMUN
#Flames1stGoal
Arsenal
#COYG
#ARSMUN
#ET302
Ethiopian Airlines
#CSW63
#ARSMUN
#Flames1stGoal
Arsenal
#COYG
Montreal
#ARSMUN
#Flames1stGoal
Arsenal
#ARSMUN
Arsenal
Ethiopian Airlines
De Gea
Edmonton
#ARSMUN
#Flames1stGoal
Arsenal
#ARSMUN
#Flames1stGoal
Arsenal
#COYG
#DaylightSavingsTime
#Canada7s
#FireBenning
#InternationalWomensDay
Arsenal
#avfc
Grealish
#bcfc
Absolutely
Liverpool
#BlackpoolAreBack
#UTMP
Arsenal
Grealish
Scotland
#C2C2019
#bhafc
#ARSMUN
#DancingOnIce
De Gea
#AfterLife
Absolutely
#ARSMUN
#DancingOnIce
#ARSMUN
Arsenal
#IREvFRA
Birmingham
#ARSMUN
Arsenal
United
#avfc
Arsenal
Grealish
Liverpool
Birmingham
#ARSMUN
#DancingOnIce
De Gea
#COYG
#DancingOnIce
#ARSMUN
Rashford
De Gea
#ARSMUN
#DancingOnIce
De Gea
#COYG
#ARSMUN
#DancingOnIce
De Gea
#COYG
#AfterLife
Liverpool
#ARSMUN
#DancingOnIce
#AfterLife
#ARSMUN
#DancingOnIce
De Gea
#ARSMUN
#DancingOnIce
De Gea
#COYG
#ARSMUN
#DancingOnIce
De Gea
#COYG
#ARSMUN
#DancingOnIce
De Gea
#COYG
#ARSMU

KeyError: 0

In [135]:
def RedditSearch(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, trends):
    for query in set(trends):
        if (int(float(ratelimit_calls)) > 0):
            #print('Before call')
            print(ratelimit_calls, ratelimit_reset)
            ratelimit_calls, ratelimit_reset = RedditQuery(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, query)  
            #print('After call')
            #print(ratelimit_calls, ratelimit_reset)
            print(type(ratelimit_calls), type(ratelimit_reset))
        else:
            print('Sleep')
            time.sleep(int(ratelimit_reset))
            ratelimit_calls, ratelimit_reset = RedditQuery(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, query)

In [129]:
def RedditQuery(reddit_headers,elastic_headers,elastic_index, ratelimit_calls, ratelimit_reset, query):
    r_get = requests.get('https://oauth.reddit.com/r/all/search?q='+query+'&t=day&limit=25', headers = reddit_headers)
    posts = r_get.json()['data']['children']
    data = []
    fields_to_keep = ['id','title', 'author', 'score', 'subreddit', 'num_comments', 'num_crossposts', 'created_utc']
    for post in posts:
        temp = {k: post['data'][k] for k in fields_to_keep}
        temp.update({'hashtag': query})
        data.append({'index': {"_index": elastic_index, '_type': '_doc', '_id': 'r_'+ temp['id'] }})
        del temp['id']
        data.append(temp)

    if data:
        data_to_post = '\n'.join(json.dumps(d) for d in data) 
        #r_post = requests.post('http://34.73.60.209:9200/'+elastic_index+'/_bulk', headers = elastic_headers, data=data_to_post+'\n')
        #print(r_post.text)
    
    print(r_get.headers['x-ratelimit-remaining'], r_get.headers['x-ratelimit-reset'])
    
    return r_get.headers['x-ratelimit-remaining'], r_get.headers['x-ratelimit-reset']

In [137]:
# Testing Enviroment for Rate Limits:
reddit_headers = getRedditHeaders()
elastic_headers = {'Content-Type':'application/json'}
elastic_index = 'test_03_10_19_pubsub'

r_get = requests.get('https://oauth.reddit.com', headers = reddit_headers)
ratelimit_calls =  r_get.headers['x-ratelimit-remaining']
ratelimit_reset = r_get.headers['x-ratelimit-reset']

RedditSearch(reddit_headers, elastic_headers, elastic_index, ratelimit_calls, ratelimit_reset, trends)

599.0 552
598.0 551
<class 'str'> <class 'str'>
598.0 551
597.0 551
<class 'str'> <class 'str'>
597.0 551
596.0 551
<class 'str'> <class 'str'>
596.0 551
595.0 551
<class 'str'> <class 'str'>
595.0 551
594.0 551
<class 'str'> <class 'str'>
594.0 551
593.0 550
<class 'str'> <class 'str'>
593.0 550
592.0 550
<class 'str'> <class 'str'>
592.0 550
591.0 550
<class 'str'> <class 'str'>
591.0 550
590.0 550
<class 'str'> <class 'str'>
590.0 550
589.0 550
<class 'str'> <class 'str'>
589.0 550
588.0 549
<class 'str'> <class 'str'>
588.0 549
587.0 549
<class 'str'> <class 'str'>
587.0 549
586.0 549
<class 'str'> <class 'str'>
586.0 549
585.0 549
<class 'str'> <class 'str'>
585.0 549
584.0 549
<class 'str'> <class 'str'>
584.0 549
583.0 548
<class 'str'> <class 'str'>
583.0 548
582.0 548
<class 'str'> <class 'str'>
582.0 548
581.0 548
<class 'str'> <class 'str'>
581.0 548
580.0 548
<class 'str'> <class 'str'>
580.0 548
579.0 548
<class 'str'> <class 'str'>
579.0 548
578.0 548
<class 'str'> <class

KeyboardInterrupt: 

In [ ]:
# This is a working example: make sure the indexes match
import json
data = [
    { "index" : { "_index" : "test5", "_type" : "type1", "_id" : "1" } },
    { "field1" : "value1" },
    { "delete" : { "_index" : "test5", "_type" : "type1", "_id" : "2" }, },
    { "create" : { "_index" : "test5", "_type" : "type1", "_id" : "3" }, },
    { "field1" : "value3" },
    { "update" : {"_id" : "1", "_type" : "type1", "_index" : "test5"} },
    { "doc" : {"field2" : "value2"} }
]

data_to_post = '\n'.join(json.dumps(d) for d in data)
print(data_to_post)
#r = requests.post('http://34.73.60.209:9200/test5/_bulk', headers = headers, data=data_to_post + '\n')
#print(r.text)

In [ ]:
# Communicating with Elastic Search:
requests.post('http://34.73.60.209:9200/hi_yash/_doc/',  json = {"user" : "kimchy",
    "post_date" : "2019-01-28T14:12:12",
    "message" : "Hi Yash"})
